# Spark HR Data Pipeline Project

## Prerequisites
1. Install required libraries and prepare spark environment

In [1]:
# Installing requuired packages
%pip install pyspark  findspark wget


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 37.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813860 sha256=aee97cb8d7b51e9964f4e003737e0a8baa8cd8c182eeeb6aaefa14cc771dee1f
  Stored in directory: /home/maishuji/.cache/pip/wheels/00/e3/92/8594f4cee2c9fd4ad82fe85e4bf2559ab8ea84ef19b1dd3d15
  Created wheel for wget: fi

In [4]:
import findspark

findspark.init()

In [ ]:
# PySpark is the Spark API for Python. We use PySpark to initialize the SparkContext.

from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

In [ ]:
# Creating a SparkContext object
sc = SparkContext.getOrCreate()

# Creating a SparkSession

spark = SparkSession.builder.appName(
    "Python Spark HR Data Pipeline Project"
).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/10 15:32:52 WARN Utils: Your hostname, maishuji, resolves to a loopback address: 127.0.1.1; using 192.168.0.14 instead (on interface wlp4s0)
25/09/10 15:32:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/10 15:32:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/10 15:32:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


2. Download the CSV data

In [ ]:
# Download the CSV data first into a local `employees.csv` file
import wget

wget.download(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/data/employees.csv"
)

'employees.csv'

## Tasks

1. Generate a Spark DataFrame from the CSV data

In [7]:
# Read data from the "employees" CSV file and import it into a DataFrame variable named "employees_df"
employees_df = spark.read.csv("employees.csv", header=True, inferSchema=True)
employees_df.show()


+------+---------+------+---+----------+
|Emp_No| Emp_Name|Salary|Age|Department|
+------+---------+------+---+----------+
|   198|   Donald|  2600| 29|        IT|
|   199|  Douglas|  2600| 34|     Sales|
|   200| Jennifer|  4400| 36| Marketing|
|   201|  Michael| 13000| 32|        IT|
|   202|      Pat|  6000| 39|        HR|
|   203|    Susan|  6500| 36| Marketing|
|   204|  Hermann| 10000| 29|   Finance|
|   205|  Shelley| 12008| 33|   Finance|
|   206|  William|  8300| 37|        IT|
|   100|   Steven| 24000| 39|        IT|
|   101|    Neena| 17000| 27|     Sales|
|   102|      Lex| 17000| 37| Marketing|
|   103|Alexander|  9000| 39| Marketing|
|   104|    Bruce|  6000| 38|        IT|
|   105|    David|  4800| 39|        IT|
|   106|    Valli|  4800| 38|     Sales|
|   107|    Diana|  4200| 35|     Sales|
|   108|    Nancy| 12008| 28|     Sales|
|   109|   Daniel|  9000| 35|        HR|
|   110|     John|  8200| 31| Marketing|
+------+---------+------+---+----------+
only showing top

2. Define q schema for the data

In [26]:
# Lets first print the inferred schema
employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- department_name: string (nullable = true)



In [27]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType,
)

# Now we define an improved schema with precise data types
employees_schema_improved = StructType(
    [
    StructField("Emp_No", IntegerType(), True),
    StructField("Emp_Name", StringType(), True),
    StructField("Salary", DoubleType(), True),  # Matches CSV header
    StructField("Age", IntegerType(), True),
    StructField("Department", StringType(), True)  # Matches CSV header
    ]
)

In [28]:
# Apply the new schema
employees_df = spark.read.csv(
    "employees.csv", schema=employees_schema_improved, header=True
)

In [29]:
# Rename columns to have consistent naming conventions
employees_df = employees_df \
    .withColumnRenamed("Emp_No", "employee_id") \
    .withColumnRenamed("Emp_Name", "employee_name") \
    .withColumnRenamed("Salary", "salary") \
    .withColumnRenamed("Department", "department_name") \
    .withColumnRenamed("Age", "age")

3. Dislay schema of DataFrame

In [30]:
employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- department_name: string (nullable = true)



4. Create a temporary view

In [31]:
# Create a temporary view so that we can run SQL queries against the DataFrame
employees_df.createOrReplaceTempView("employees")

5. Execute an SQL query

In [32]:
# SQL query to fetch only the records from the view where the age is greater than 30
sql_query = "SELECT * FROM employees WHERE age > 30"
result_df = spark.sql(sql_query)
result_df.show()

+-----------+-------------+-------+---+---------------+
|employee_id|employee_name| salary|age|department_name|
+-----------+-------------+-------+---+---------------+
|        199|      Douglas| 2600.0| 34|          Sales|
|        200|     Jennifer| 4400.0| 36|      Marketing|
|        201|      Michael|13000.0| 32|             IT|
|        202|          Pat| 6000.0| 39|             HR|
|        203|        Susan| 6500.0| 36|      Marketing|
|        205|      Shelley|12008.0| 33|        Finance|
|        206|      William| 8300.0| 37|             IT|
|        100|       Steven|24000.0| 39|             IT|
|        102|          Lex|17000.0| 37|      Marketing|
|        103|    Alexander| 9000.0| 39|      Marketing|
|        104|        Bruce| 6000.0| 38|             IT|
|        105|        David| 4800.0| 39|             IT|
|        106|        Valli| 4800.0| 38|          Sales|
|        107|        Diana| 4200.0| 35|          Sales|
|        109|       Daniel| 9000.0| 35|         

6. Calculate Average Salary by Department

In [33]:
sql_query_avg_salary = """
SELECT department_name, AVG(salary) AS average_salary
FROM employees
GROUP BY department_name
ORDER BY average_salary DESC
"""
avg_salary_df = spark.sql(sql_query_avg_salary)
avg_salary_df.show()

+---------------+-----------------+
|department_name|   average_salary|
+---------------+-----------------+
|             IT|           7400.0|
|      Marketing|6633.333333333333|
|             HR|           5837.5|
|        Finance|           5730.8|
|          Sales|5492.923076923077|
+---------------+-----------------+



7. Filter to select records from the IT departement

In [34]:
employees_df.filter(employees_df.department_name == "IT").show()

+-----------+-------------+-------+---+---------------+
|employee_id|employee_name| salary|age|department_name|
+-----------+-------------+-------+---+---------------+
|        198|       Donald| 2600.0| 29|             IT|
|        201|      Michael|13000.0| 32|             IT|
|        206|      William| 8300.0| 37|             IT|
|        100|       Steven|24000.0| 39|             IT|
|        104|        Bruce| 6000.0| 38|             IT|
|        105|        David| 4800.0| 39|             IT|
|        111|       Ismael| 7700.0| 32|             IT|
|        129|        Laura| 3300.0| 38|             IT|
|        132|           TJ| 2100.0| 34|             IT|
|        136|        Hazel| 2200.0| 29|             IT|
+-----------+-------------+-------+---+---------------+



8. Add 10% Bonus to Salaries ( with new column `salary_after_bonus`)

In [36]:
from pyspark.sql.functions import col

employees_df = employees_df.withColumn("salary_after_bonus", col("salary") * 1.1)
employees_df.show()

+-----------+-------------+-------+---+---------------+------------------+
|employee_id|employee_name| salary|age|department_name|salary_after_bonus|
+-----------+-------------+-------+---+---------------+------------------+
|        198|       Donald| 2600.0| 29|             IT|2860.0000000000005|
|        199|      Douglas| 2600.0| 34|          Sales|2860.0000000000005|
|        200|     Jennifer| 4400.0| 36|      Marketing|            4840.0|
|        201|      Michael|13000.0| 32|             IT|14300.000000000002|
|        202|          Pat| 6000.0| 39|             HR| 6600.000000000001|
|        203|        Susan| 6500.0| 36|      Marketing| 7150.000000000001|
|        204|      Hermann|10000.0| 29|        Finance|           11000.0|
|        205|      Shelley|12008.0| 33|        Finance|13208.800000000001|
|        206|      William| 8300.0| 37|             IT|            9130.0|
|        100|       Steven|24000.0| 39|             IT|26400.000000000004|
|        101|        Neen

9. Find Maximum Salary by Age

In [40]:
from pyspark.sql.functions import max
employees_df.groupBy("age").agg(max("salary")).show()

+---+-----------+
|age|max(salary)|
+---+-----------+
| 31|     8200.0|
| 34|     7800.0|
| 28|    12008.0|
| 27|    17000.0|
| 26|     3600.0|
| 37|    17000.0|
| 35|     9000.0|
| 39|    24000.0|
| 38|     6000.0|
| 29|    10000.0|
| 32|    13000.0|
| 33|    12008.0|
| 30|     8000.0|
| 36|     7900.0|
+---+-----------+



10. Self-Join on Employee Data

In [ ]:
joined_df = employees_df.join(employees_df, col("employees.employee_id") == col("employees.employee_id"), "inner")

{"ts": "2025-09-10 16:36:50.838", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `employees`.`employee_id` cannot be resolved. Did you mean one of the following? [`employee_id`, `employee_id`, `employee_name`, `employee_name`, `salary_after_bonus`]. SQLSTATE: 42703", "context": {"file": "java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o145.join.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `employees`.`employee_id` cannot be resolved. Did you mean one of the following? [`employee_id`, `employee_id`, `employee_name`, `employee_name`, `salary_after_bonus`]. SQLSTATE: 42703;\n'Join Inner, '`=`('employees.e

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `employees`.`employee_id` cannot be resolved. Did you mean one of the following? [`employee_id`, `employee_id`, `employee_name`, `employee_name`, `salary_after_bonus`]. SQLSTATE: 42703;
'Join Inner, '`=`('employees.employee_id, 'employees.age)
:- Project [employee_id#140, employee_name#141, salary#142, age#144, department_name#143, (salary#142 * 1.1) AS salary_after_bonus#231]
:  +- Project [employee_id#140, employee_name#141, salary#142, Age#137 AS age#144, department_name#143]
:     +- Project [employee_id#140, employee_name#141, salary#142, Age#137, Department#138 AS department_name#143]
:        +- Project [employee_id#140, employee_name#141, Salary#136 AS salary#142, Age#137, Department#138]
:           +- Project [employee_id#140, Emp_Name#135 AS employee_name#141, Salary#136, Age#137, Department#138]
:              +- Project [Emp_No#134 AS employee_id#140, Emp_Name#135, Sa

11. Calculate Average Employee Age

In [ ]:
from pyspark.sql.functions import avg 
employees_df.agg(avg(col("age")).alias("average_age")).show()

+-----------+
|average_age|
+-----------+
|      33.56|
+-----------+



12. Calculate Total Salary By Department

In [45]:
from pyspark.sql.functions import sum 

salary_per_department_df = employees_df.groupBy("department_name").agg(sum("salary"))
salary_per_department_df.show()

+---------------+-----------+
|department_name|sum(salary)|
+---------------+-----------+
|          Sales|    71408.0|
|             HR|    46700.0|
|        Finance|    57308.0|
|      Marketing|    59700.0|
|             IT|    74000.0|
+---------------+-----------+



13. Sort Data by Age and Salary

In [47]:
employees_df.sort(col("age").asc(), col("salary").desc()).show()

+-----------+-------------+-------+---+---------------+------------------+
|employee_id|employee_name| salary|age|department_name|salary_after_bonus|
+-----------+-------------+-------+---+---------------+------------------+
|        137|       Renske| 3600.0| 26|      Marketing|3960.0000000000005|
|        101|        Neena|17000.0| 27|          Sales|           18700.0|
|        114|          Den|11000.0| 27|        Finance|12100.000000000002|
|        108|        Nancy|12008.0| 28|          Sales|13208.800000000001|
|        130|        Mozhe| 2800.0| 28|      Marketing|3080.0000000000005|
|        126|        Irene| 2700.0| 28|             HR|2970.0000000000005|
|        204|      Hermann|10000.0| 29|        Finance|           11000.0|
|        115|    Alexander| 3100.0| 29|        Finance|3410.0000000000005|
|        134|      Michael| 2900.0| 29|          Sales|3190.0000000000005|
|        198|       Donald| 2600.0| 29|             IT|2860.0000000000005|
|        140|       Joshu

14. Count Employees in Each Department

In [48]:
from pyspark.sql.functions import count
employees_df.groupBy("department_name").agg(count("employee_id").alias("employee_count")).show()

+---------------+--------------+
|department_name|employee_count|
+---------------+--------------+
|          Sales|            13|
|             HR|             8|
|        Finance|            10|
|      Marketing|             9|
|             IT|            10|
+---------------+--------------+



15. Filter Employees with the letter o in the Name

In [49]:
employees_df.filter(col("employee_name").contains("o")).show()

+-----------+-------------+------+---+---------------+------------------+
|employee_id|employee_name|salary|age|department_name|salary_after_bonus|
+-----------+-------------+------+---+---------------+------------------+
|        198|       Donald|2600.0| 29|             IT|2860.0000000000005|
|        199|      Douglas|2600.0| 34|          Sales|2860.0000000000005|
|        110|         John|8200.0| 31|      Marketing|            9020.0|
|        112|  Jose Manuel|7800.0| 34|             HR|            8580.0|
|        130|        Mozhe|2800.0| 28|      Marketing|3080.0000000000005|
|        133|        Jason|3300.0| 38|          Sales|3630.0000000000005|
|        139|         John|2700.0| 36|          Sales|2970.0000000000005|
|        140|       Joshua|2500.0| 29|        Finance|            2750.0|
+-----------+-------------+------+---+---------------+------------------+

